In [6]:
import pyspark

In [7]:
pyspark.__file__

'/home/abyssde232024/spark/spark-3.3.2-bin-hadoop3/python/pyspark/__init__.py'

In [8]:
from pyspark.sql import SparkSession

In [9]:
print(SparkSession.getActiveSession())

None


In [53]:
spark_sql.stop()

In [10]:
spark_sql = SparkSession.builder \
    .master("local[*]") \
    .appName('dezoomcamp2024') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/06 20:11:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
df_green = spark_sql.read.parquet('data/processed/green/*/*', header=True)

In [12]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-26 12:45:44|  2020-01-26 12:45:45|                 N|         5|          25|         264|              1|          .00|      40.05|    0|      

In [13]:
df_green = df_green.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [14]:
df_green.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- extra: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- ehail_fee: string (nullable = true)
 |-- improvement_surcharge: string (nullable = true)
 |-- total_amount: string (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: string (nullable = true)



In [15]:
df_green.registerTempTable('green_taxi_trips')

/home/abyssde232024/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [16]:
df_green_revenue = spark_sql.sql('''
                        SELECT 
                              date_trunc("hour", pickup_datetime) as hour, 
                              PULocationID,
                              SUM(total_amount) as amount,
                              COUNT(1) as record_number
                        FROM green_taxi_trips
                        where pickup_datetime >= "2020-01-01 00:00:00"
                        GROUP BY 1, 2
                        --ORDER BY 1, 2
                        ''')

In [17]:
df_green_revenue.show()

+-------------------+------------+------------------+-------------+
|               hour|PULocationID|            amount|record_number|
+-------------------+------------+------------------+-------------+
|2020-01-22 19:00:00|          65| 657.0299999999999|           41|
|2020-01-14 10:00:00|         243|141.92000000000002|            5|
|2020-01-31 11:00:00|         220|168.07999999999998|            6|
|2020-01-29 09:00:00|         116|            302.97|           19|
|2020-01-19 10:00:00|         241|             90.49|            2|
|2020-01-02 13:00:00|          74|1037.6299999999992|           71|
|2020-01-20 07:00:00|          82|             72.36|            6|
|2020-01-11 11:00:00|          83|             54.06|            5|
|2020-01-11 18:00:00|          41|476.01000000000016|           45|
|2020-01-06 21:00:00|         129|288.02000000000004|           14|
|2020-01-27 14:00:00|          51|            153.13|            3|
|2020-01-01 01:00:00|          17|            59

In [18]:
df_green_revenue.repartition(20).write.parquet('data/report/revenue/green', mode='overwrite')

In [19]:
df_yellow = spark_sql.read.parquet('data/processed/yellow/*/*', header=True)

In [20]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [21]:
df_yellow = df_yellow.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [22]:
df_yellow.registerTempTable('yellow_taxi_trips')

In [23]:
df_yellow_revenue = spark_sql.sql('''
                        SELECT 
                              date_trunc("hour", pickup_datetime) as hour, 
                              PULocationID,
                              SUM(total_amount) as amount,
                              COUNT(1) as record_number
                        FROM yellow_taxi_trips
                        where pickup_datetime >= "2020-01-01 00:00:00"
                        GROUP BY 1, 2
                        --ORDER BY 1, 2
                        ''')

In [24]:
df_yellow_revenue.repartition(20).write.parquet('data/report/revenue/yellow', mode='overwrite')

In [25]:
df_yellow_revenue.show()

+-------------------+------------+------------------+-------------+
|               hour|PULocationID|            amount|record_number|
+-------------------+------------+------------------+-------------+
|2020-01-10 06:00:00|         144|            149.51|            9|
|2020-01-29 03:00:00|          13|            248.89|            6|
|2020-01-13 21:00:00|         255|            108.98|            8|
|2020-01-04 12:00:00|         249| 2245.569999999999|          152|
|2020-01-02 15:00:00|         236| 7382.910000000018|          504|
|2020-01-31 21:00:00|          50|1784.4399999999994|          104|
|2020-01-07 12:00:00|          68| 3046.229999999997|          181|
|2020-01-02 09:00:00|         161|4299.9299999999985|          242|
|2020-01-23 17:00:00|         233| 4642.769999999999|          235|
|2020-01-06 22:00:00|         158| 923.2100000000003|           57|
|2020-01-15 07:00:00|          13|2432.6299999999997|          118|
|2020-01-23 19:00:00|         224|            32

In [26]:
df_green_revenue_temp = df_green_revenue.withColumnRenamed('amount', 'amount_green') \
                                        .withColumnRenamed('record_number', 'record_number_green')
df_yellow_revenue_temp = df_yellow_revenue.withColumnRenamed('amount', 'amount_yellow') \
                                        .withColumnRenamed('record_number', 'record_number_yellow')

In [27]:
df_join = df_green_revenue_temp.join(df_yellow_revenue_temp, on=['hour', 'PULocationID'], how='outer').withColumnRenamed('PULocationID', 'zone')

In [28]:
df_join.show()

+-------------------+----+------------------+-------------------+------------------+--------------------+
|               hour|zone|      amount_green|record_number_green|     amount_yellow|record_number_yellow|
+-------------------+----+------------------+-------------------+------------------+--------------------+
|2020-01-01 00:00:00|   3|              null|               null|              25.0|                   1|
|2020-01-01 00:00:00|   4|              null|               null|1004.3000000000002|                  57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                 45| 455.1700000000001|                  38|
|2020-01-01 00:00:00|  12|              null|               null|             107.0|                   6|
|2020-01-01 00:00:00|  37|175.66999999999996|                  6|161.60999999999999|                   7|
|2020-01-01 00:00:00|  40|            168.98|                  8|             89.97|                   5|
|2020-01-01 00:00:00|  45|              null| 

In [29]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [35]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, LongType, DoubleType, IntegerType

In [39]:
lookup_schema = StructType([
    StructField("LocationID", IntegerType(), nullable=True),
    StructField("Borough", StringType(), nullable=True),
    StructField("Zone", StringType(), nullable=True),
    StructField("service_zone", StringType(), nullable=True)
])

In [40]:
df_lookup = spark_sql.read \
            .option("header", "true") \
            .schema(lookup_schema) \
            .csv('taxi_zone_lookup.csv')

In [41]:
df_lookup.write.parquet('data/lookup', mode='overwrite')

In [46]:
df_lookup = spark_sql.read.parquet('data/lookup', header=True).withColumnRenamed('Zone', 'Zone_name').withColumnRenamed('LocationID', 'zone')
            

In [47]:
df_lookup.show()

+----+-------------+--------------------+------------+
|zone|      Borough|           Zone_name|service_zone|
+----+-------------+--------------------+------------+
|   1|          EWR|      Newark Airport|         EWR|
|   2|       Queens|         Jamaica Bay|   Boro Zone|
|   3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|   4|    Manhattan|       Alphabet City| Yellow Zone|
|   5|Staten Island|       Arden Heights|   Boro Zone|
|   6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|   7|       Queens|             Astoria|   Boro Zone|
|   8|       Queens|        Astoria Park|   Boro Zone|
|   9|       Queens|          Auburndale|   Boro Zone|
|  10|       Queens|        Baisley Park|   Boro Zone|
|  11|     Brooklyn|          Bath Beach|   Boro Zone|
|  12|    Manhattan|        Battery Park| Yellow Zone|
|  13|    Manhattan|   Battery Park City| Yellow Zone|
|  14|     Brooklyn|           Bay Ridge|   Boro Zone|
|  15|       Queens|Bay Terrace/Fort ...|   Boro Zone|
|  16|    

In [48]:
df_join = df_join.join(df_lookup, on='zone', how='left')

In [49]:
df_join.show()

+----+-------------------+------------------+-------------------+------------------+--------------------+---------+--------------------+------------+
|zone|               hour|      amount_green|record_number_green|     amount_yellow|record_number_yellow|  Borough|           Zone_name|service_zone|
+----+-------------------+------------------+-------------------+------------------+--------------------+---------+--------------------+------------+
|   3|2020-01-01 00:00:00|              null|               null|              25.0|                   1|    Bronx|Allerton/Pelham G...|   Boro Zone|
|   4|2020-01-01 00:00:00|              null|               null|1004.3000000000002|                  57|Manhattan|       Alphabet City| Yellow Zone|
|   7|2020-01-01 00:00:00| 769.7299999999996|                 45| 455.1700000000001|                  38|   Queens|             Astoria|   Boro Zone|
|  12|2020-01-01 00:00:00|              null|               null|             107.0|                

In [52]:
df_join.repartition(20).write.parquet('data/report/revenue/total_with_lookup', mode='overwrite')